### Full Range

策略： 在full range内提供流动性

In [157]:
import sys
sys.path.append("..") 
import pandas as pd
from datetime import datetime
from importlib import reload
from poolData import swapData
from utils import utils
reload(swapData)
reload(utils)

query = swapData.SwapDataQuery()
utils = utils.utils()

In [210]:
# position info
# 选定池子： token0: usdc(decimal: 6), token1: weth(decimal: 18)
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
query.query_positions(pool_id=pool_id, block_gte=16859000, limit=10, orderBy="owner")
decimal0 = 6
decimal1 = 18
# 初始头寸的美元价值
initial_position = 100000000000

# 选定时间
begin = datetime(2023, 3, 19)
end = datetime.now()

pool_id: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 - type: <class 'str'>


In [211]:
# 获取起始时间和结束时间的价格
# 假设本金是1000刀，token0是usdc，数量是500
token0_amount = initial_position / 2
# 计算token1数量的时候需要知道价格
# 获取价格
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640")
liquidity_data
price_begin = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][0]))
price_end = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][liquidity_data.shape[0]-1]))
price_begin_real = utils.price2RealPrice(price_begin, decimal0=decimal0, decimal1=decimal1)
price_end_real = utils.price2RealPrice(price_end, decimal0=decimal0, decimal1=decimal1)
# 拿到价格之后才可以计算初始头寸
token1_amount = token0_amount * price_begin_real

this is the cursor: 1682496000


### 计算该策略的手续费

首先需要计算出头寸的liquidity是多少。

full range策略相当于v2中的头寸，在全价格范围内提供流动性，在v3中就是minTick和maxTick(0,infinity)之间提供流动性。

我在测试网上提供了一个full range的[头寸](https://goerli.etherscan.io/tx/0x199a0a53afbaa7c6b542dc2bdc2766639e9250055032238ebcf204d7375c8aa5), 得到：

tickLower： -887270
tickUpper： 887270


In [186]:
# 计算价格范围
tickLower = -887270
tickUpper = 887270
price_low = utils.tickIndex2Price(tickLower)
price_upper = utils.tickIndex2Price(tickUpper)
print("this is the lower price: ", price_low)
print("this is the upper price: ", price_upper)

this is the lower price:  2.939544628365392e-39
this is the upper price:  3.4018874568203963e+38


In [187]:
token1_amount

27.904097419391654

In [212]:
# 计算liquidity
l = (token0_amount * token1_amount) ** 0.5
print("l_fullRange:", l)

l_fullRange: 1181187906.7149236


In [197]:
# 获取选定时间期限内的所有swaps
swaps = query.query_swaps(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

this is the cursor: 0x0118cefd9bf44fd4c4752f6c36bed9f54a1cb535863bac7a20e0e77b73e5a346#4616514
this is the cursor: 0x0239410d5d6db9472a06fcd3a8752ec19f819cf9555372e36ee63e16f8ba18e1#4779330
this is the cursor: 0x034c1fa5b3c0798a709aeb7690b9509475573979fe98d6f9ac9e0619946358bd#4825991
this is the cursor: 0x047e9d01bb9c61542ef93e9b3b8273c50ca297f3d1dde12a96affd3c45b159bd#4624078
this is the cursor: 0x05a107cd9eb359e834d339cc2526e11f0f1105be2c2dc04c232079b157d9a6c6#4763420
this is the cursor: 0x06cf47009747f2d53efe1294b2d06acb73f2657ff29c80235c86bd927deca9a3#4631896
this is the cursor: 0x07ea376e73affef1c63adb22bc7a7cc2aa0c388457f02a015821193f0ac8aabe#4615726
this is the cursor: 0x090b81bbb2000da0773ac211d1c0ecef867698b98604904be061bf20833a6405#4620425
this is the cursor: 0x0a2dd6e3b5d070df6c728ac920fcd989ab86b60115c72c81d3ad094c6f55920d#4724425
this is the cursor: 0x0b4eb31e9cc18d5633f4bf9c8c2c5746220ec9c83c4b0ea8e8e6c491d88fbf2b#4711726
this is the cursor: 0x0c6f2b2c27bed92ece30fe81778d

In [202]:
# 计算fee
# convert str into float
swaps['amount0'] = swaps['amount0'].astype(float)
swaps['amount1'] = swaps['amount1'].astype(float)
swaps['amountUSD'] = swaps['amountUSD'].astype(float)
swaps['sqrtPriceX96'] = swaps['sqrtPriceX96'].astype(float)
swaps['tick'] = swaps['tick'].astype(int)
swaps['logIndex'] = swaps['logIndex'].astype(int)
# transform X86 into normal number
swaps["sqrtPriceX96"] = (swaps["sqrtPriceX96"] / (2**96) ) ** 2
# transform sqrt_p into p
swaps["sqrtPriceX96"] = 1 / (swaps["sqrtPriceX96"] * (10 ** 12))

In [203]:
# 现在来加总然后计算手续费
swaps1 = swaps
# 先把不用的列去掉
swaps1 = swaps1.drop(swaps1.columns[[0, 2, 3, 4, 7, 10]], axis = 1)
swaps1.head(6)

,timestamp,amount0,amount1,sqrtPriceX96,tick
0,1679680919,-19922.526611,11.442382,0.0,201692
1,1680218951,1104.872406,-0.617085,0.0,201422
2,1681392455,-3951.498533,1.973163,0.0,200293
3,1679774651,-12033.180287,7.000000,0.0,201820
4,1681089035,3299.547442,-1.774376,0.0,201044
5,1679404235,-5000.197259,2.750358,0.0,201260


In [204]:
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

this is the cursor: 1682492400


In [205]:
liquidity_data

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1679158800,31691854789759819627,1871666948661873416199899917784088,1791.851542392230339967547914533666,0.0005580819483878331967604985081167192,201410,2314975469199928457298700227744982,1111162292302232016625192018264273618935405,362517939.1226720089381000671278629,0,0,0,0,372,1798.390894813620679474180985197866,1803.236399827628569085607995221095,1769.953587075568576406795835267567,1791.851542392230339967547914533666
1,1679162400,32915185914729732691,1867893400320714844515651086096830,1799.098710137099104209582500255936,0.0005558338707962253356429190522474716,201369,2315038215475779131181192686558395,1111189202831556094328153678234461295869361,363811618.6075028876192885997227013,0,0,0,0,366,1790.640320160079804025064362917098,1799.108571408040728063168297875355,1786.686354081499257894742581324593,1799.098710137099104209582500255936
2,1679166000,34369179026466248085,1869482743679436927327266292647007,1796.040997899866322473028892703993,0.0005567801632419932350116030494839081,201386,2315126854003039415933596971303595,1111241826769754789231620166343530092470985,363967492.8611761592527996744286012,0,0,0,0,333,1799.099225334904656531075427168004,1809.622901299084675474781496124223,1794.120699231514675114167039962852,1796.040997899866322473028892703993
3,1679169600,34183163223601833767,1868781276195693273308669229878293,1797.389578204559120182790388692096,0.000556362411424970994583196746389777,201379,2315168855482994101747218874079807,1111263525863761589158339866419647898430929,363205815.9317762712257394030005428,0,0,0,0,317,1796.054690522833853305120372567828,1805.57781837591635090288239389413,1792.725482994450340228488201862034,1797.389578204559120182790388692096
4,1679173200,34246952167731560476,1875752556320845172108591732547304,1784.054320332921814271944303424757,0.0005605210495011106585938907612211571,201453,2315193497548537858877306471738285,1111292420021385719852093400706179840724576,360909125.4200933323216227809336806,0,0,0,0,309,1797.199358810234823401563218695319,1797.220201287329445413990798860576,1783.038903387140753714413070004309,1784.054320332921814271944303424757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,1682478000,19974979606699332511,1835372892322969109796219789737537,1863.419094246594727004202941704458,0.0005366479301878750900885372077673467,201018,2355695442961685604614522080873221,1132960063115952527584494123687189484829090,457065964.4101575501522156817956537,0,0,0,0,125,1868.954415820080373305980695949018,1871.900844803519252023462198932213,1863.376103131048204848023208472717,1863.419094246594727004202941704458
923,1682481600,20079147173112138077,1833960800239411179743007578180333,1866.289747062648700644089862715904,0.0005358224796411697898453848275011378,201003,2355706790584998477054301180596396,1132963092509937365686070118858962573492301,456944150.7537582028931504489000487,0,0,0,0,146,1863.419094246594727004202941704458,1868.079158671586189587692733108823,1863.419094246594727004202941704458,1866.289747062648700644089862715904
924,1682485200,21099513284256475236,1832212390766229784081927199736230,1869.853302888126439066099105002248,0.000534801312196751579708054479268464,200984,2355721979099424391375588433074317,1132967572498394079568605140306372402023176,456946116.674010652148630961695913,0,0,0,0,154,1866.719859584542724539011713101046,1870.18957843412616640366826963631,1866.719859584542724539011713101046,1869.853302888126439066099105002248
925,1682488800,20082828377118460290,1834322954389487956409484348537620,1865.552888970253552918333422518791,0.0005360341193821522958527974158965844,201007,2355742416033158008569947010732748,1132982937267843117937972397755462415131951,456943740.5379049725979322965388548,0,0,0,0,186,1869.839920863508924503792223728627,1873.228427954734676867648340056136

In [206]:
liquidity_data = liquidity_data[["periodStartUnix", "liquidity"]]
liquidity_data = liquidity_data.rename(columns={"periodStartUnix": "timestamp", "liquidity": "liquidity"})
# 先排序
swaps["timestamp"] = swaps["timestamp"].astype(int)
swaps = swaps.sort_values(by="timestamp")
liquidity_data = liquidity_data.sort_values(by="timestamp")
# 找liquidity data只时间最接近的来合并
merged_data = pd.merge_asof(
    swaps,
    liquidity_data[["timestamp", "liquidity"]],
    on="timestamp",
    direction="nearest"
)
merged_data.head(3)

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex,liquidity
0,0x0861d00acafcd74202e82741c031db7417fbca4c45fe...,1679155223,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,-17615.760520,9.835104,17613.789058,0.0,201409,17,31691854789759819627
1,0x3da2883fd656b389ddce46a794901afc9f0d68ec3c3a...,1679155295,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x3392ef365f255d735a0bd0b83ebca213ba7b4fc4,-256165.425339,143.200000,256297.538930,0.0,201421,13,31691854789759819627
2,0x2a3a91acf0e9406df421ecdbd847f39343e043c649b7...,1679155343,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x274767ad3ad1be357981934f7d8d3d39e8886c95,176.130000,-0.098407,176.174145,0.0,201426,246,31691854789759819627


In [217]:
# 获取头寸的liquidity，后续手续费的分成是按照头寸的liquidity占总liquidity的比例来计算的
position_liquidity = l
merged_data['liquidity'] = merged_data['liquidity'].astype(float)
merged_data['ratio'] = float(position_liquidity) / merged_data["liquidity"]
merged_data['amount0'] = merged_data['amount0'].astype(float).apply(lambda x: max(0, x))
merged_data['amount1'] = merged_data['amount1'].astype(float).apply(lambda x: max(0, x))

merged_data["fee0"] = merged_data["amount0"] * merged_data['ratio'] * 0.0005
fee0_sum = merged_data["fee0"].sum()

merged_data["fee1"] = merged_data["amount1"] * merged_data['ratio'] * 0.0005
fee1_sum = merged_data["fee1"].sum()

print(fee0_sum)
print(fee1_sum)

4.118737633490241e-05
2.2100453272677043e-08


*** 

以下是用invest metric来计算投资指标的内容

In [218]:
from investMetric import metric
reload(metric)

<module 'investMetric.metric' from '/Users/bcfh/Documents/GitHub/learn_uniswapv3/strategies/../investMetric/metric.py'>

In [219]:
begin = datetime(2023, 3, 19)
end = datetime.now()
days = (end - begin).days
days

38

In [220]:
merged_data["fee0_sum"] = merged_data['fee0'].cumsum()
merged_data["fee1_sum"] = merged_data['fee1'].cumsum()
# transform X86 into normal number
merged_data["real_price"] = merged_data["tick"].apply(lambda x: 1/ (utils.price2RealPrice(utils.tickIndex2Price(merged_data["tick"][0]), decimal0=6, decimal1=18)))
merged_data["position_value"] = (token0_amount  + merged_data["fee0_sum"]) + (token1_amount + merged_data["fee1_sum"]) * merged_data["real_price"]
merged_data["position_value0"] = (token0_amount  + merged_data["fee0_sum"])
merged_data["position_value1"] = (token1_amount + merged_data["fee1_sum"]) * merged_data["real_price"]
volatility = metric.volatility(merged_data["position_value"])
roi = metric.roi(merged_data["position_value"][0], merged_data["position_value"].iloc[-1], (end - begin).days)
print("** volatility = ", volatility)

rf = 0.017 # risk free rate 先随便写一个
sharpe_ratio = metric.sharpRatio(roi, volatility, rf)

** volatility =  3.573848603017842e-05


In [221]:
metric.metric(merged_data=merged_data, token0_amount=token0_amount, token1_amount=token1_amount, begin=begin, end=end, decimal0=decimal0, decimal1=decimal1, rf=rf)

** volatility =  3.573848603017842e-05
** roi =  (9.154708793593892e-16, 8.43769498715119e-15)
** sharpe_ratio =  [-475.6776766 -475.6776766]
